# Installing Keycloak

## Prerequisites
- [Learning Jupyter](https://github.com/NephTek/Public_Notebooks/blob/main/getting-started/learning_jupyter.ipynb)
- Install Jupyter bash kernel:
  - [Windows](https://github.com/NephTek/Public_Notebooks/blob/main/getting-started/windows/install_jupyter_bash_kernel.ipynb)
  - [Mac](https://github.com/NephTek/Public_Notebooks/blob/main/getting-started/mac/install_jupyter_bash_kernel.ipynb)
  - [Linux](https://github.com/NephTek/Public_Notebooks/blob/main/getting-started/linux/install_jupyter_bash_kernel.ipynb)
- [Install Kubernetes](https://github.com/NephTek/Public_Notebooks/blob/main/kubernetes/install_k8s_cluster.ipynb)
- [Install DNSmasq](https://github.com/NephTek/Public_Notebooks/blob/main/dnsmasq/install_dnsmasq.ipynb)
- [Install cert-manager](https://github.com/NephTek/Public_Notebooks/blob/main/cert-manager/install_cert-manager.ipynb)
- [Install Monitoring](https://github.com/NephTek/Public_Notebooks/blob/main/monitoring/install_kube-prometheus-stack.ipynb)
- [Install Logging](https://github.com/NephTek/Public_Notebooks/blob/main/logging/install_loki.ipynb)
- [Install Ingress](https://github.com/NephTek/Public_Notebooks/blob/main/ingress/install_ingress-nginx.ipynb)
- [Install Postgres](https://github.com/NephTek/Public_Notebooks/blob/main/postgres/install_postgres.ipynb)
- [Install OpenLDAP](https://github.com/NephTek/Public_Notebooks/blob/main/openldap/install_openldap.ipynb)

## Install Keycloak Operator
The Keycloak operator creates several new CRDs in Kubernetes to simplify the process of managing Keycloak and configurations.  

### Clone the keycloak-operator project
Start with cloning the keycloak-operator project locally:

In [1]:
git clone https://github.com/keycloak/keycloak-operator.git

Cloning into 'keycloak-operator'...
remote: Enumerating objects: 12631, done.
remote: Counting objects: 100% (2386/2386), done.
remote: Compressing objects: 100% (413/413), done.
remote: Total 12631 (delta 2120), reused 1995 (delta 1967), pack-reused 10245
Receiving objects: 100% (12631/12631), 11.41 MiB | 1.02 MiB/s, done.
Resolving deltas: 100% (7226/7226), done.


: 1

### Apply the operator manifests
We have built a kustomization overlay that installs the Keycloak operator, pinning the operator version number to v17.0.0 and changing the Keycloak image to an ARM64-architecture docker image of v17.0.0 of Keycloak.  Install the operator using `kubectl` and its built-in Kustomize functionality: 

In [2]:
kubectl apply -k kustomization/overlays/rancher-desktop

namespace/keycloak unchanged
customresourcedefinition.apiextensions.k8s.io/keycloakbackups.keycloak.org unchanged
customresourcedefinition.apiextensions.k8s.io/keycloakclients.keycloak.org unchanged
customresourcedefinition.apiextensions.k8s.io/keycloakrealms.keycloak.org unchanged
customresourcedefinition.apiextensions.k8s.io/keycloaks.keycloak.org unchanged
customresourcedefinition.apiextensions.k8s.io/keycloakusers.keycloak.org unchanged
serviceaccount/keycloak-operator unchanged
role.rbac.authorization.k8s.io/keycloak-operator unchanged
rolebinding.rbac.authorization.k8s.io/keycloak-operator unchanged
deployment.apps/keycloak-operator unchanged


: 1

### Install the Postgres database
Using the Zalando Postgres operator installed in a previous step, we install a Postgres database instance for Keycloak:

In [5]:
kubectl apply -f keycloak-postgres.yaml -n keycloak

postgresql.acid.zalan.do/acid-keycloak-cluster created


Next, we extract the value of the Postgres password into a secret that our Keycloak instance will use: 

In [2]:
PGPASSWORD=$(kubectl get secret keycloak.acid-keycloak-cluster.credentials.postgresql.acid.zalan.do -n keycloak -o 'jsonpath={.data.password}' | base64 -d)

In [3]:
kubectl create secret generic keycloak-db-secret -n keycloak \
        --from-literal=POSTGRES_DATABASE=keycloak \
        --from-literal=POSTGRES_EXTERNAL_ADDRESS=acid-keycloak-cluster.keycloak.svc.cluster.local \
        --from-literal=POSTGRES_EXTERNAL_PORT=5432 \
        --from-literal=POSTGRES_USERNAME=keycloak \
        --from-literal=POSTGRES_PASSWORD=$PGPASSWORD

secret/keycloak-db-secret created


In [4]:
kubectl label secret keycloak-db-secret app=keycloak -n keycloak

secret/keycloak-db-secret labeled


In [ ]:
With this secret, we now can create our Keycloak instance using the `keycloak` CRD:

In [15]:
kubectl apply -f keycloak-sso.yaml -n keycloak

keycloak.keycloak.org/keycloak-sso created


### Verify installation

In [16]:
kubectl get pods -n keycloak

NAME                                 READY   STATUS    RESTARTS        AGE
keycloak-operator-846f9d566f-xzhlm   1/1     Running   4 (4h17m ago)   21d
acid-keycloak-cluster-0              1/1     Running   0               16m
keycloak-0                           0/1     Running   0               7s


### Create Ingress 
The Ingress object exposes our Keycloak instance at the URL: https://keycloak.k8s.nephtek.com
This Ingress object also creates and manages a TLS certificate using cert-manager, installed in a prior step.

In [5]:
kubectl apply -f keycloak-ingress.yaml -n keycloak

ingressroutetcp.traefik.containo.us/keycloak unchanged


### Create Keycloak Realm for LDAP

In [4]:
kubectl apply -f keycloak-realm.yaml -n keycloak

keycloakrealm.keycloak.org/ldap-realm configured


: 1

### Finished

### Next Steps
